# Table with all pages in the main namespace resolving redirects


## Starting spark session

In [1]:
import os, sys
import datetime
import calendar
import time
import string
import random

import findspark
findspark.init('/usr/lib/spark2')
from pyspark.sql import SparkSession
from pyspark.sql import functions as F, types as T, Window
import wmfdata.spark as wmfspark

## defining the spark session
spark_config = {}
spark = wmfspark.get_session(
    app_name='Pyspark notebook', 
    type='regular'
#     extra_settings=spark_config
)
spark

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


# Generating the table with pages and their redirects

We get a table with all main namespace articles:
- wiki_db
- page_id
- page_title
- page_is_redirect,
- page_id_resolved,
- page_title_resolved
    
    
Page-titles contain underscores!!!

In [2]:
## selecting the snapshot and wikidb
snapshot = "2022-01"

In [3]:
## running the query and saving the table

## get a list of all wikipedia-projects (e.g. not wikidata)
df_projects = (
    spark.read.table('wmf_raw.mediawiki_project_namespace_map')
    .where(F.col("snapshot") == snapshot)
    .where(F.col("hostname").contains("wikipedia"))
    .select(F.col("dbname").alias("wiki_db"))
    .distinct()
)

df_pages = (
    ## select table
    spark.read.table('wmf_raw.mediawiki_page')
    ## select snapshot
    .where( F.col('snapshot') == snapshot )
    ## filter wikipedias
    .join(
        df_projects,
        on = "wiki_db",
        how = "inner"
    )
    
    ## main namespace
    .where(F.col('page_namespace') == 0 )
    .select(
        "wiki_db",
        'page_id',
        'page_title',
        'page_is_redirect',
    )
)
df_redirect = (
    ## select table
    spark.read.table('wmf_raw.mediawiki_redirect')
    ## select snapshot
    .where( F.col('snapshot') == snapshot )
    ## filter wikipedias
    .join(
        df_projects,
        on = "wiki_db",
        how = "inner"
    )
    ## main namespace
    .where(F.col('rd_namespace') == 0 )
    .select(
        F.col("wiki_db"),
        F.col('rd_from').alias('page_id_from'),
        F.col('rd_title').alias('page_title_to')
    )
)

## wiki_db, page_id, page_title, page_is_redirect, page_id_resolved, page_title_resolved
df_pages_resolved = (
    df_pages
    ## join the redirect-table: pid_from --> page_title_to
    ## this adds a new column "page_title_to" for all pages that a redirect
    .join(
        df_redirect.withColumnRenamed("page_id_from","page_id"),
        on = ["wiki_db","page_id"],
        how = "left"
    )
    ## create a new column page_title_resolved
    .withColumn('page_title_resolved', F.coalesce(F.col('page_title_to'),F.col('page_title')) )
    ## join the page-table to get page-ids from titles (not resolved)
    .join(
        (df_pages
         .withColumnRenamed("page_title","page_title_resolved")
         .withColumnRenamed("page_id","page_id_resolved")
         .withColumnRenamed("page_is_redirect","page_is_redirect_resolved")
        ),
        on = ["wiki_db","page_title_resolved"],
        how = "left"
    )
    ## only keep pages that are not a redirect after resolving anymore
    .where(F.col("page_is_redirect_resolved")==False)
    .select(
        "wiki_db",
        "page_id",
        "page_title",
        "page_is_redirect",
        "page_id_resolved",
        "page_title_resolved",
    )
)

fname_save = "pages-titles-resolved_all-wikis_%s"%(snapshot)
df_pages_resolved.write.mode("overwrite").parquet(fname_save)

# Check result

### Load the table

In [4]:
snapshot = "2022-01"
fname_read = "pages-titles-resolved_all-wikis_%s"%(snapshot)
df = spark.read.parquet(fname_read)
df.show()

+-------+-------+--------------------+----------------+----------------+--------------------+
|wiki_db|page_id|          page_title|page_is_redirect|page_id_resolved| page_title_resolved|
+-------+-------+--------------------+----------------+----------------+--------------------+
| abwiki|   4337|                  37|           false|            4337|                  37|
| abwiki|   9724|                Хәба|            true|            4243|     5_(ахыԥхьаӡара)|
| abwiki|  18565|     5_(ахыҧхьаӡара)|            true|            4243|     5_(ахыԥхьаӡара)|
| abwiki|   6676|                Хуба|            true|            4243|     5_(ахыԥхьаӡара)|
| abwiki|   4243|     5_(ахыԥхьаӡара)|           false|            4243|     5_(ахыԥхьаӡара)|
| abwiki|   8702|                 743|           false|            8702|                 743|
| abwiki|   8766|                 807|           false|            8766|                 807|
| abwiki|  18099|                   Χ|           false|     

In [5]:
## total number of pages across all wikis
df.count()

107084049

### check the enwiki-table

In [6]:
df_en = df.where(F.col("wiki_db")=="enwiki")
df_en.show()

+-------+--------+--------------------+----------------+----------------+--------------------+
|wiki_db| page_id|          page_title|page_is_redirect|page_id_resolved| page_title_resolved|
+-------+--------+--------------------+----------------+----------------+--------------------+
| enwiki|55029148|!!!_(disambiguation)|           false|        55029148|!!!_(disambiguation)|
| enwiki|12359213|   "Bufo"_scorteccii|           false|        12359213|   "Bufo"_scorteccii|
| enwiki|66740517|    Scortecci's_toad|            true|        12359213|   "Bufo"_scorteccii|
| enwiki|67240880|     Bufo_scorteccii|            true|        12359213|   "Bufo"_scorteccii|
| enwiki|66740701|Duttaphrynus_scor...|            true|        12359213|   "Bufo"_scorteccii|
| enwiki|49353393|`Abd_ar-Rahman_ib...|            true|        31708220|'Abd_ar-Rahman_ib...|
| enwiki|31708220|'Abd_ar-Rahman_ib...|           false|        31708220|'Abd_ar-Rahman_ib...|
| enwiki|31707369|Abd_ar-Rahman_ibn...|           

In [7]:
## number of non-redirects
df_en.where(F.col("page_is_redirect")==False).count()

6447170

In [8]:
## number of redirects
df_en.where(F.col("page_is_redirect")==True).count()

9915464